In [18]:
import pandas as pd
import requests,json
from sklearn import metrics
from toolkits.nlp import pre_cor_circ

In [19]:
def get_server_res(data, url, col_name):
    '''
    服务器接口测试程序
    传入 dict, 传出 DataFrame
    '''
    # data = {'record':[{'id':0,'title':'ss','content':'zzz'},]}
    # data = {"record":marked_human_data.iloc[:5,:3].to_dict(orient = 'records')}
    # url "http://47.93.77.19:10000/correlation_negative"
    headers={'content-type':'application/json'}
    result = requests.post(url,
                      data = json.dumps(data),
                      headers=headers, allow_redirects=True)
    # print(result.text)
    json_data = json.loads(result.text)
    parse_data = []
    elapsed_time = json_data['elapsed_time']
    for i in range(len(json_data['docs'])):
        parse_data.append([json_data['docs'][i]['id'],
                          json_data['docs'][i][col_name]])
    parse_data = pd.DataFrame(parse_data, columns = ['id', col_name])    
    return parse_data, elapsed_time

In [24]:
fiel_path = r'circ_data_tendency\20181112\circ_tendency_feedback_20181116.xlsx'

combined_data_1 = pd.read_excel(fiel_path, '0')
combined_data_2 = pd.read_excel(fiel_path, '-1')
combined_data = pd.concat([combined_data_1, combined_data_2])
combined_data['title_content'] = combined_data['title']+'。'+combined_data['content']

In [25]:
from sklearn.externals import joblib
pipeline = joblib.load("model/cbrc_chapter_tendency_1114.pkl.z")

title_content = pre_cor_circ.handle_contents(combined_data['title_content'].tolist())
local_label = pipeline.predict(title_content)
combined_data['local_label'] = local_label

print(combined_data.shape)
combined_data.head()

(400, 10)


D:\software\conda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,id,group_id,predict_label,tendency,label,title,content,publishtime,title_content,local_label
0,11967202,微信,消费服务,0,0,车辆被撞对方耍赖怎么办？看这里彻底帮你搞定！| 保险与你,来源：财险互动 如果开车正常行驶被别人撞了，对方全责，找对方赔偿就好。 可事情并不总是那么简...,2018-11-08 23:55:00,车辆被撞对方耍赖怎么办？看这里彻底帮你搞定！| 保险与你。来源：财险互动 如果开车正常行驶被...,-1
1,11915584,微博,资本市场,0,0,【板王快讯：中国平安取代贝莱德 成汇丰控股第一大股东】在最近一次增持后，中国平安已取代贝莱德...,【板王快讯：中国平安取代贝莱德 成汇丰控股第一大股东】在最近一次增持后，中国平安已取代贝莱德...,2018-11-06 18:41:26,【板王快讯：中国平安取代贝莱德 成汇丰控股第一大股东】在最近一次增持后，中国平安已取代贝莱德...,-1
2,11936030,新闻,消费服务,0,0,旅责险统保示范项目显成效,阳光产险30万元的旅游意外保险赔偿金于近日送达山东邹平孙某某的遗属。旅行社责任保险统保示...,2018-11-07 16:46:41,旅责险统保示范项目显成效。 阳光产险30万元的旅游意外保险赔偿金于近日送达山东邹平孙某某的...,-1
3,11902803,新闻,公司内部管理,0,0,香港金融科技潮起 累计募资达9.4亿美元,（原标题：香港金融科技潮起）香港金融局在金融科技周上宣布，新推出的贸易融资数码化平台——“贸...,2018-11-06 07:21:27,香港金融科技潮起 累计募资达9.4亿美元。（原标题：香港金融科技潮起）香港金融局在金融科技周...,0
4,11959788,微信,产品销售,0,0,如何像挑选口红一样为自己配置保险,今天转载一篇同事Margaret的原创文章， 日常使用的口红，是要经过挑选才能在成百上千的试...,2018-11-08 17:25:58,如何像挑选口红一样为自己配置保险。今天转载一篇同事Margaret的原创文章， 日常使用的口...,0


In [28]:
# y_pred_class = pipeline.predict(X_test)
y_pred_class = tend_data['local_label'].tolist()
y_test = tend_data['label'].tolist()
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.4175
             precision    recall  f1-score   support

         -1       0.17      0.69      0.27        64
          0       0.86      0.37      0.51       336

avg / total       0.75      0.42      0.48       400

confusion_matrix: 
[[ 44  20]
 [213 123]]


In [26]:
# combined_data['id'] = range(combined_data.shape[0])
combined_data['title'] = combined_data['title'].astype(str) 
combined_data['content'] = combined_data['content'].astype(str)
data = {"record":combined_data.loc[:,['id', 'title' ,'content']].to_dict(orient = 'records')}
url = "http://47.93.77.19:10000/tendency_analysis_i"
col_name = 'tendency'
parse_data, elapsed_time = get_server_res(data, url, col_name)
parse_data.columns = ['id', 'online_label']
print(parse_data.shape)
parse_data.head()

(400, 2)


,id,online_label
0,11967202,-1
1,11915584,0
2,11936030,0
3,11902803,0
4,11959788,0


In [27]:
tend_data = pd.merge(parse_data, combined_data, on = 'id', how = 'inner')
print(tend_data.shape)
tend_data.head()

(400, 11)


,id,online_label,group_id,predict_label,tendency,label,title,content,publishtime,title_content,local_label
0,11967202,-1,微信,消费服务,0,0,车辆被撞对方耍赖怎么办？看这里彻底帮你搞定！| 保险与你,来源：财险互动 如果开车正常行驶被别人撞了，对方全责，找对方赔偿就好。 可事情并不总是那么简...,2018-11-08 23:55:00,车辆被撞对方耍赖怎么办？看这里彻底帮你搞定！| 保险与你。来源：财险互动 如果开车正常行驶被...,-1
1,11915584,0,微博,资本市场,0,0,【板王快讯：中国平安取代贝莱德 成汇丰控股第一大股东】在最近一次增持后，中国平安已取代贝莱德...,【板王快讯：中国平安取代贝莱德 成汇丰控股第一大股东】在最近一次增持后，中国平安已取代贝莱德...,2018-11-06 18:41:26,【板王快讯：中国平安取代贝莱德 成汇丰控股第一大股东】在最近一次增持后，中国平安已取代贝莱德...,-1
2,11936030,0,新闻,消费服务,0,0,旅责险统保示范项目显成效,阳光产险30万元的旅游意外保险赔偿金于近日送达山东邹平孙某某的遗属。旅行社责任保险统保示...,2018-11-07 16:46:41,旅责险统保示范项目显成效。 阳光产险30万元的旅游意外保险赔偿金于近日送达山东邹平孙某某的...,-1
3,11902803,0,新闻,公司内部管理,0,0,香港金融科技潮起 累计募资达9.4亿美元,（原标题：香港金融科技潮起）香港金融局在金融科技周上宣布，新推出的贸易融资数码化平台——“贸...,2018-11-06 07:21:27,香港金融科技潮起 累计募资达9.4亿美元。（原标题：香港金融科技潮起）香港金融局在金融科技周...,0
4,11959788,0,微信,产品销售,0,0,如何像挑选口红一样为自己配置保险,今天转载一篇同事Margaret的原创文章， 日常使用的口红，是要经过挑选才能在成百上千的试...,2018-11-08 17:25:58,如何像挑选口红一样为自己配置保险。今天转载一篇同事Margaret的原创文章， 日常使用的口...,0


In [29]:
# y_pred_class = pipeline.predict(X_test)
y_pred_class = tend_data['online_label'].tolist()
y_test = tend_data['label'].tolist()
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.6325
             precision    recall  f1-score   support

         -1       0.27      0.75      0.40        64
          0       0.93      0.61      0.74       336

avg / total       0.82      0.63      0.68       400

confusion_matrix: 
[[ 48  16]
 [131 205]]
